# Building a Production dbt Project: Week-by-Week Guide

This notebook walks through building a **production-grade dbt + DuckDB analytics pipeline** step by step.
Each week builds on the previous one, progressing from a blank project to a fully-tested pipeline with
18 models, 91 tests, snapshots, and CI/CD.

| Week | Focus | What You Build |
|------|-------|----------------|
| 1 | Project Setup & First Models | `dbt init`, profiles, `ref()`, example models |
| 2 | Raw Data & Reference Tables | Download data, seeds, sources, data exploration |
| 3 | The Staging Layer | Clean, rename, filter, surrogate keys |
| 4 | Testing & Data Quality | Generic tests, unit tests, custom macros |
| 5 | Intermediate & Dimensional Modeling | Metrics, dimensions, facts, analytics marts |
| 6 | Advanced & Production Readiness | Incremental, Python models, exports, snapshots, CI/CD |

**The dataset**: NYC Yellow Taxi trips (January 2024, ~3M rows).
But the patterns apply to **any dataset** -- each week includes tips on adapting the concepts to your own data.

---

## Prerequisites

Make sure you've run the full build first:
```bash
uv sync
uv run python scripts/download_data.py
uv run python scripts/setup_project.py
cd nyc_taxi_dbt && uv run dbt deps && uv run dbt build --full-refresh --profiles-dir .
```

This notebook connects to the DuckDB database that dbt built and explores every layer of the pipeline.

## Setup: Connect to Our DuckDB Warehouse

In [1]:
import duckdb
import os
from pathlib import Path

# Connect to the dbt-managed database
DB_PATH = Path("dev.duckdb")
assert DB_PATH.exists(), "Run 'make build' first to create the database"

con = duckdb.connect(str(DB_PATH), read_only=True)

# Helper to run queries and display results
def q(sql):
    """Run a SQL query and return a pandas DataFrame."""
    return con.execute(sql).fetchdf()

# What schemas did dbt create?
q("""
    SELECT table_schema, count(*) as tables
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
    GROUP BY table_schema
    ORDER BY table_schema
""")

,table_schema,tables
0,main,2
1,main_intermediate,3
2,main_marts,10
3,main_raw,3
4,main_staging,4
5,snapshots,1


dbt organized our data into **separate schemas by layer**:
- `main` -- Week 1 example models (starter models that live in the default schema)
- `main_raw` -- Seed data (CSV reference tables loaded by `dbt seed`)
- `main_staging` -- Cleaned, renamed versions of raw data
- `main_intermediate` -- Calculated metrics and aggregations
- `main_marts` -- Business-ready facts, dimensions, and analytics
- `snapshots` -- SCD Type 2 historical tracking

This separation is a **dbt best practice**. Each layer has a single responsibility, making the pipeline easy to debug and maintain.
The schema names follow the pattern `<target>_<custom>` (e.g., `main` + `staging` = `main_staging`).

---

## Week 1: Project Setup & First Models

### Goals
- Initialize a dbt project with DuckDB
- Understand `profiles.yml` vs `dbt_project.yml`
- Write your first models and use `ref()`
- Understand materializations (view vs table)

### What to Build
```bash
# 1. Set up Python dependencies
uv init && uv add dbt-core dbt-duckdb duckdb

# 2. Scaffold the dbt project
uv run dbt init my_project

# 3. Configure profiles.yml for DuckDB (project-local)
# 4. Create two simple models: one with hardcoded data, one using ref()

# 5. Run it
cd my_project && uv run dbt run --profiles-dir .
```

### Key Files Created
```
my_project/
  dbt_project.yml      # WHAT the project is (name, paths, materializations)
  profiles.yml          # WHERE to connect (DuckDB path, threads, schema)
  models/
    example/
      my_first_model.sql    # Hardcoded data -- verify pipeline works
      my_second_model.sql   # Uses ref() -- learn DAG dependencies
```

In [2]:
# Week 1 models: simple hardcoded data + a ref() downstream model
print("=== my_first_model (hardcoded taxi trips) ===")
display(q("SELECT * FROM main.my_first_model"))

print("\n=== my_second_model (adds trip_category via ref()) ===")
display(q("SELECT * FROM main.my_second_model"))

print("\nNotice: my_second_model adds a 'trip_category' column computed from fare_amount.")
print("It references my_first_model using {{ ref('my_first_model') }}.")
print("dbt automatically builds my_first_model BEFORE my_second_model.")

=== my_first_model (hardcoded taxi trips) ===


,trip_id,taxi_type,fare_amount,pickup_datetime
0,1,yellow,12.50,2024-01-15 08:30:00
1,2,yellow,8.75,2024-01-15 14:15:00
2,3,yellow,25.00,2024-01-16 19:45:00



=== my_second_model (adds trip_category via ref()) ===


,trip_id,taxi_type,fare_amount,pickup_datetime,trip_category
0,1,yellow,12.50,2024-01-15 08:30:00,medium
1,2,yellow,8.75,2024-01-15 14:15:00,short
2,3,yellow,25.00,2024-01-16 19:45:00,long



Notice: my_second_model adds a 'trip_category' column computed from fare_amount.
It references my_first_model using {{ ref('my_first_model') }}.
dbt automatically builds my_first_model BEFORE my_second_model.


### Key Concepts

**`ref()`**: The foundation of dbt. Instead of hardcoding table names, you write `{{ ref('model_name') }}`.
dbt uses these references to:
1. Build a **DAG** (Directed Acyclic Graph) of dependencies
2. Automatically run models in the correct order
3. Handle schema/database changes transparently

**Materializations**:
| Type | Creates | Build Speed | Query Speed | Best For |
|------|---------|-------------|-------------|----------|
| `view` | SQL view | Fast | Re-executes SQL each time | Staging, intermediate |
| `table` | Physical table | Slower | Fast (pre-computed) | Marts, final outputs |

**Two config files**:
| File | Purpose | Controls |
|------|---------|----------|
| `profiles.yml` | WHERE to connect | Database type, path, credentials |
| `dbt_project.yml` | WHAT the project is | Name, model paths, default materializations |

### Applying Week 1 to Any Dataset

1. **Start with `dbt init`** -- it scaffolds the entire project structure for you
2. **Use project-local `profiles.yml`** with `--profiles-dir .` so the config is portable and versionable
3. **Begin with hardcoded data** in your first model to verify the pipeline works end-to-end before loading real data
4. **Adapter choice matters**:
   - DuckDB: local/dev, zero infrastructure, handles millions of rows on a laptop
   - Postgres: small teams, shared access
   - Snowflake/BigQuery/Databricks: production scale, cloud-native
5. **Use `uv`** (or `poetry`) instead of raw pip -- reproducible dependency management saves debugging time
6. **Create a Makefile** early to wrap common `dbt` commands -- your team will thank you

---

## Week 2: Raw Data & Reference Tables

### Goals
- Download and explore your real dataset
- Load small reference tables as dbt seeds (CSVs)
- Configure external sources for large data files
- Understand seeds vs sources vs `read_parquet()`

### What to Build
```bash
# 1. Download data
uv run python scripts/download_data.py

# 2. Place lookup CSVs in seeds/ directory
# 3. Define external sources in models/sources.yml
# 4. Run seeds
cd nyc_taxi_dbt && uv run dbt seed --profiles-dir .
```

### Thought Process

Before writing any dbt models, you need to **understand your data**. The NYC Taxi & Limousine Commission
publishes trip records monthly as parquet files. January 2024 has ~3 million yellow taxi trips.

**Why parquet?** Column-oriented, compressed, includes schema metadata. DuckDB reads it natively without importing.

**Why DuckDB?** Zero infrastructure. No Docker, no server, no credentials. Just a single file.
It handles 3M rows in under 2 seconds on a laptop.

In [3]:
# Let's peek at the raw parquet file directly
raw_stats = q("""
    SELECT
        count(*) as total_rows,
        min(tpep_pickup_datetime) as earliest_pickup,
        max(tpep_pickup_datetime) as latest_pickup,
        count(DISTINCT PULocationID) as unique_pickup_zones,
        count(DISTINCT DOLocationID) as unique_dropoff_zones
    FROM read_parquet('data/yellow_tripdata_2024-01.parquet')
""")
print(f"Raw parquet: {raw_stats['total_rows'][0]:,} rows")
print(f"Date range: {raw_stats['earliest_pickup'][0]} to {raw_stats['latest_pickup'][0]}")
print(f"Unique zones: {raw_stats['unique_pickup_zones'][0]} pickup, {raw_stats['unique_dropoff_zones'][0]} dropoff")

Raw parquet: 2,964,624 rows
Date range: 2002-12-31 22:59:39 to 2024-02-01 00:01:15
Unique zones: 260 pickup, 261 dropoff


In [4]:
# What does the raw data look like? (column names are messy!)
q("SELECT * FROM read_parquet('data/yellow_tripdata_2024-01.parquet') LIMIT 3")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0


**Problems we can already see:**
- Column names are inconsistent: `VendorID`, `PULocationID`, `tpep_pickup_datetime`, `Airport_fee` (mixed case)
- Location IDs are just numbers -- meaningless without a lookup table
- Payment types are coded (1, 2, 3...) -- not human-readable
- Financial amounts need rounding to 2 decimal places
- There might be null values, negative fares, or out-of-range dates

This is exactly what the **staging layer** (Week 3) will fix. But first, we need reference data.

In [5]:
# Seed tables -- small reference CSVs loaded by dbt seed
print("=== Payment Types ===")
display(q("SELECT * FROM main_raw.payment_type_lookup ORDER BY payment_type_id"))

print("\n=== Rate Codes ===")
display(q("SELECT * FROM main_raw.rate_code_lookup ORDER BY rate_code_id"))

print("\n=== Taxi Zones (sample) ===")
display(q("SELECT * FROM main_raw.taxi_zone_lookup ORDER BY LocationID LIMIT 10"))

=== Payment Types ===


,payment_type_id,payment_type_name
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip



=== Rate Codes ===


,rate_code_id,rate_code_name
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride
6,99,Unknown



=== Taxi Zones (sample) ===


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


### Seeds vs Sources: When to Use Which

| Method | Best For | Loaded By | Size Limit |
|--------|----------|-----------|------------|
| **Seeds** (CSV in `seeds/`) | Small reference data (<10K rows) | `dbt seed` | Small (committed to git) |
| **Sources** (parquet/external) | Large raw datasets | DuckDB reads directly | Unlimited (gitignored) |
| `read_parquet()` | Quick prototyping | DuckDB function in SQL | Unlimited |

We use seeds for lookup tables (265 zones, 6 payment types, 7 rate codes) and a source for the 3M-row trip data.

**Source configuration** (in `sources.yml`):
```yaml
sources:
  - name: raw_nyc_taxi
    tables:
      - name: raw_yellow_trips
        config:
          external_location: "read_parquet('../data/yellow_tripdata_2024-01.parquet')"
```

**Important for dbt-duckdb**: Use `config:` (not `meta:`) for `external_location`. This is a common gotcha.

### Applying Week 2 to Any Dataset

1. **Explore before modeling**: Always `SELECT *` from your raw data first. Note column names, types, nulls, ranges.
2. **Identify reference/lookup tables**: Any coded values (status codes, type IDs, region IDs) need a lookup CSV
3. **Seeds go in git, data does not**: Seeds are small CSVs that belong in version control. Raw data files (parquet, CSV dumps) go in `.gitignore`
4. **Document your sources**: `sources.yml` tracks lineage -- dbt knows where your data comes from
5. **Source freshness**: Add `loaded_at_field` and `freshness` config so dbt can warn when source data goes stale
6. **Create a download script**: Automate data acquisition (`scripts/download_data.py`) so anyone can reproduce your setup

---

## Week 3: The Staging Layer

### Goals
- Build staging models that clean raw data
- Rename columns to consistent `snake_case`
- Cast types explicitly and filter bad data
- Generate surrogate keys for deduplication
- Install dbt packages (`dbt_utils`)

### What to Build
```bash
# 1. Add packages.yml with dbt_utils
# 2. Install packages
cd nyc_taxi_dbt && uv run dbt deps --profiles-dir .

# 3. Create staging models: stg_yellow_trips, stg_taxi_zones, etc.
# 4. Build and test
uv run dbt build --select staging --profiles-dir .
```

### The Staging Pattern

The staging layer is a **contract between raw data and the rest of the pipeline**. Its jobs:
1. **Rename** columns to consistent `snake_case`
2. **Cast** types explicitly (don't trust implicit casting)
3. **Filter** obviously bad data (nulls in required fields, negative fares)
4. **Generate** surrogate keys for deduplication

The rule: **one staging model per source**. Every downstream model references staging, never raw data directly.

```sql
-- Key parts of stg_yellow_trips.sql
select
    -- Surrogate key for deduplication
    {{ dbt_utils.generate_surrogate_key([
        'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'PULocationID', 'DOLocationID', 'fare_amount', 'total_amount'
    ]) }} as trip_id,

    -- Consistent naming + explicit casting
    cast("VendorID" as integer) as vendor_id,
    cast("PULocationID" as integer) as pickup_location_id,
    round(cast(fare_amount as decimal(10, 2)), 2) as fare_amount,
    ...
from source
where tpep_pickup_datetime is not null    -- filter bad data
  and fare_amount >= 0                     -- no negative fares
  and cast(tpep_pickup_datetime as date) >= date '2024-01-01'
```

In [6]:
# How many rows survived staging filters?
raw_count = q("SELECT count(*) as n FROM read_parquet('data/yellow_tripdata_2024-01.parquet')")['n'][0]
staged_count = q("SELECT count(*) as n FROM main_staging.stg_yellow_trips")['n'][0]
filtered = raw_count - staged_count

print(f"Raw rows:     {raw_count:>12,}")
print(f"After staging:{staged_count:>12,}")
print(f"Filtered out: {filtered:>12,} ({filtered/raw_count*100:.2f}%)")

Raw rows:        2,964,624
After staging:   2,927,159
Filtered out:       37,465 (1.26%)


In [7]:
# Clean column names -- compare raw vs staged
print("=== Raw Columns ===")
raw_cols = q("SELECT column_name FROM (DESCRIBE SELECT * FROM read_parquet('data/yellow_tripdata_2024-01.parquet'))")
print(", ".join(raw_cols['column_name'].tolist()))

print("\n=== Staged Columns ===")
staged_cols = q("SELECT column_name FROM information_schema.columns WHERE table_schema='main_staging' AND table_name='stg_yellow_trips' ORDER BY ordinal_position")
print(", ".join(staged_cols['column_name'].tolist()))

=== Raw Columns ===
VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, Airport_fee

=== Staged Columns ===
trip_id, vendor_id, rate_code_id, pickup_location_id, dropoff_location_id, payment_type_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance_miles, store_and_fwd_flag, fare_amount, extra_amount, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee


### Why Include fare_amount + total_amount in the Surrogate Key?

NYC taxi data has **adjustment records** -- the same vendor, same timestamps, same locations,
but different fare amounts (one original, one corrected). Without including financial columns
in the key, these would collide and we'd lose data. This is a real-world data quality issue
you only discover by testing.

### Data Quality Findings
- ~37K rows with negative fares (adjustment records) -- **filtered out** in staging
- ~18 rows with dates outside January 2024 -- **filtered out** in staging
- ~140K rows with null passenger_count -- **kept** (non-critical field, `severity: warn`)
- Location IDs 264/265 exist in trips but not in zone lookup -- **handled** with `severity: warn`
- VendorID=6 exists (~260 rows) -- **added** to accepted_values

### Applying Week 3 to Any Dataset

**Staging checklist for any source:**
1. **Rename** all columns to `snake_case` -- consistency prevents downstream confusion
2. **Cast** every column explicitly -- don't rely on implicit type inference
3. **Generate a surrogate key** if no natural primary key exists (most raw data doesn't have one)
4. **Include ALL disambiguating columns** in the surrogate key -- test for uniqueness to verify
5. **Filter only obvious garbage** (nulls in required fields, negative amounts, impossible dates)
6. **Don't over-filter** -- keep questionable data and use test severity `warn` for known issues
7. **Install dbt_utils early** -- `generate_surrogate_key`, `date_spine`, and test helpers save time

```bash
# packages.yml
packages:
  - package: dbt-labs/dbt_utils
    version: [">= 1.1.0", "< 2.0.0"]
```

---

## Week 4: Testing & Data Quality

### Goals
- Add generic tests (not_null, unique, accepted_values, relationships)
- Write custom macros and custom generic tests
- Write unit tests with mock data
- Write singular tests (custom SQL)
- Understand test severity and when to `warn` vs `error`

### What to Build
```bash
# 1. Create schema YAML files with test definitions
# 2. Create custom macros (duration_minutes, test_positive_value)
# 3. Create singular tests (assert_fare_not_exceeds_total.sql)
# 4. Create unit tests (_unit_tests.yml)

# Run all tests
cd nyc_taxi_dbt && uv run dbt test --profiles-dir .

# Run only unit tests
uv run dbt test --select "test_type:unit" --profiles-dir .
```

### The Testing Pyramid

```
         /\
        /  \  Singular tests (custom SQL returning failing rows)
       /    \
      /------\  Custom generic tests (reusable macro tests)
     /        \
    /----------\  dbt_utils tests (accepted_range, etc.)
   /            \
  /--------------\  Built-in generic tests (not_null, unique, accepted_values, relationships)
```

**Our project: 91 tests total** (84 data tests + 7 unit tests).

In [8]:
# Let's verify key data quality expectations manually
print("=== Data Quality Checks ===")

# 1. No null trip IDs
null_ids = q("SELECT count(*) as n FROM main_staging.stg_yellow_trips WHERE trip_id IS NULL")['n'][0]
print(f"Null trip_ids in staging: {null_ids} (expected: 0)")

# 2. Trip IDs are unique
total = q("SELECT count(*) as n FROM main_staging.stg_yellow_trips")['n'][0]
distinct = q("SELECT count(DISTINCT trip_id) as n FROM main_staging.stg_yellow_trips")['n'][0]
print(f"Total rows: {total:,}, Distinct trip_ids: {distinct:,}, Duplicates: {total - distinct}")

# 3. All trip durations are positive (after intermediate filtering)
neg_dur = q("SELECT count(*) as n FROM main_intermediate.int_trip_metrics WHERE trip_duration_minutes < 0")['n'][0]
print(f"Negative durations: {neg_dur} (expected: 0)")

# 4. All fares are non-negative
neg_fares = q("SELECT count(*) as n FROM main_staging.stg_yellow_trips WHERE fare_amount < 0")['n'][0]
print(f"Negative fares in staging: {neg_fares} (expected: 0 -- filtered in staging)")

# 5. Mart contracts -- check column types
contracts = q("""
    SELECT table_name, count(*) as columns
    FROM information_schema.columns
    WHERE table_schema = 'main_marts'
    GROUP BY table_name
    ORDER BY table_name
""")
print(f"\nMart models with enforced contracts:")
for _, row in contracts.iterrows():
    print(f"  {row['table_name']}: {row['columns']} columns")

=== Data Quality Checks ===
Null trip_ids in staging: 0 (expected: 0)
Total rows: 2,927,159, Distinct trip_ids: 2,927,159, Duplicates: 0
Negative durations: 0 (expected: 0)
Negative fares in staging: 0 (expected: 0 -- filtered in staging)

Mart models with enforced contracts:
  anomaly_daily_trips: 12 columns
  dim_dates: 10 columns
  dim_locations: 4 columns
  dim_payment_types: 2 columns
  export_daily_revenue: 18 columns
  fct_trips: 31 columns
  fct_trips__dbt_backup: 31 columns
  mart_daily_revenue: 18 columns
  mart_hourly_demand: 8 columns
  mart_location_performance: 12 columns


### Unit Tests: Testing Logic Without Real Data

Unit tests are powerful because they test **transformation logic in isolation**.
You provide mock inputs and assert expected outputs.

Example from our project -- testing that the staging model renames columns correctly:

```yaml
# _unit_tests.yml (staging)
unit_tests:
  - name: test_stg_trips_renames_and_casts
    model: stg_yellow_trips
    given:
      - input: source('raw_nyc_taxi', 'raw_yellow_trips')
        format: sql  # needed for dbt-duckdb external sources
        rows: |
          SELECT 2 AS "VendorID", 161 AS "PULocationID", ...
    expect:
      rows:
        - {vendor_id: 2, pickup_location_id: 161, ...}
```

**Key gotchas:**
- For dbt-duckdb external sources, use `format: sql` for mock inputs (the source doesn't exist as a physical table)
- For incremental models, add `overrides: { macros: { is_incremental: false } }`
- dbt 1.11+ requires `arguments:` wrapper for generic tests with arguments

### Custom Macros

Our project includes three macros:
- **`duration_minutes(start, end)`** -- Calculates time difference in minutes
- **`cents_to_dollars(col, precision)`** -- Converts cents to dollars with rounding
- **`test_positive_value(model, column_name)`** -- Custom generic test: asserts column >= 0

### Applying Week 4 to Any Dataset

1. **Start with generic tests**: `not_null` and `unique` on every primary key, `accepted_values` on coded fields
2. **Add relationship tests**: Verify foreign keys between your models (`relationships` test)
3. **Use test severity wisely**:
   - `error` (default): Things that should **never** happen (null PKs, duplicate keys)
   - `warn`: Known data quality issues in external data you can't control
4. **Write unit tests for complex logic**: Mock inputs + expected outputs catch transformation bugs
5. **Create custom generic tests** for domain-specific rules (e.g., `test_positive_value`)
6. **Write singular tests** for cross-column validations (e.g., fare <= total_amount)
7. **Run tests with every build**: Use `dbt build` (not `dbt run`) -- it seeds, runs, AND tests in one command

```yaml
# Test severity example in schema.yml
columns:
  - name: passenger_count
    tests:
      - not_null:
          severity: warn  # Known issue: ~140K nulls in source data
```

---

## Week 5: Intermediate Layer & Dimensional Modeling

### Goals
- Build intermediate models with calculated metrics
- Understand dimensional modeling (Kimball methodology)
- Build fact tables, dimension tables, and analytics marts
- Add model contracts for schema enforcement

### What to Build
```bash
# 1. Create intermediate models: int_trip_metrics, int_daily_summary, int_hourly_patterns
# 2. Create dimension tables: dim_locations, dim_dates, dim_payment_types
# 3. Create fact table: fct_trips (joins metrics + dimensions)
# 4. Create analytics marts: mart_daily_revenue, mart_location_performance, mart_hourly_demand

cd nyc_taxi_dbt && uv run dbt build --profiles-dir .
```

### The Intermediate Layer

The intermediate layer does **transformations that multiple downstream models need**.
Instead of calculating trip duration in 5 different places, we calculate it once here.

`int_trip_metrics` adds:
- `trip_duration_minutes` -- using custom macro: `{{ duration_minutes('pickup_datetime', 'dropoff_datetime') }}`
- `avg_speed_mph` -- distance / (duration / 60)
- `cost_per_mile` -- fare / distance
- `tip_percentage` -- tip / fare * 100
- Time dimensions: `pickup_date`, `pickup_hour`, `pickup_day_of_week`, `is_weekend`

It also **filters impossible trips**: duration < 1 minute, > 12 hours, or speed > 100 mph.

In [9]:
# Row counts through the pipeline
counts = q("""
    SELECT
        (SELECT count(*) FROM main_staging.stg_yellow_trips) as staging,
        (SELECT count(*) FROM main_intermediate.int_trip_metrics) as intermediate,
        (SELECT count(*) FROM main_marts.fct_trips) as marts
""")

stg = counts['staging'][0]
intm = counts['intermediate'][0]
mart = counts['marts'][0]

print(f"Staging:      {stg:>12,} rows")
print(f"Intermediate: {intm:>12,} rows  (filtered {stg - intm:,} impossible trips)")
print(f"Marts:        {mart:>12,} rows  (1:1 with intermediate -- joins don't add/remove rows)")

Staging:         2,927,159 rows
Intermediate:    2,904,047 rows  (filtered 23,112 impossible trips)
Marts:           2,904,047 rows  (1:1 with intermediate -- joins don't add/remove rows)


In [10]:
# What does the enriched data look like?
q("""
    SELECT
        trip_id,
        pickup_date,
        pickup_hour,
        pickup_day_of_week,
        is_weekend,
        trip_duration_minutes,
        trip_distance_miles,
        round(avg_speed_mph, 1) as avg_speed_mph,
        fare_amount,
        tip_amount,
        round(tip_percentage, 1) as tip_pct,
        round(cost_per_mile, 2) as cost_per_mile
    FROM main_intermediate.int_trip_metrics
    WHERE trip_distance_miles > 0
    LIMIT 10
""")

,trip_id,pickup_date,pickup_hour,pickup_day_of_week,is_weekend,trip_duration_minutes,trip_distance_miles,avg_speed_mph,fare_amount,tip_amount,tip_pct,cost_per_mile
0,386743c8daec06772dbbf232931fe1cd,2024-01-01,0,Monday,False,20,1.72,5.2,17.7,0.00,0.0,10.29
1,474005d712a129710bb0c0f9c0b3e541,2024-01-01,0,Monday,False,6,1.80,18.0,10.0,3.75,37.5,5.56
2,9865c0f037c8f981c22cb9d702c4fabf,2024-01-01,0,Monday,False,18,4.70,15.7,23.3,3.00,12.9,4.96
3,0b9e2825d0a475497ec7ce76822885c9,2024-01-01,0,Monday,False,8,1.40,10.5,10.0,2.00,20.0,7.14
4,e7ad070018e90d61819c37bb9a14c1f1,2024-01-01,0,Monday,False,6,0.80,8.0,7.9,3.20,40.5,9.88
5,b0326ba8a0581e05c7c4e9056d25a6b8,2024-01-01,0,Monday,False,32,4.70,8.8,29.6,6.90,23.3,6.30
6,551054a8e9723e2e4ede64d044f621e0,2024-01-01,0,Monday,False,26,10.82,25.0,45.7,10.00,21.9,4.22
7,fe7045aef395d9a8347b31ee9c3aa290,2024-01-01,0,Monday,False,28,3.00,6.4,25.4,0.00,0.0,8.47
8,3efdb40f7be3a19d33e56cc9a1ee49be,2024-01-01,0,Monday,False,28,5.44,11.7,31.0,0.00,0.0,5.70
9,88fa8de28efbcaaa91bf99e0eb01d260,2024-01-01,0,Monday,False,1,0.04,2.4,3.0,0.00,0.0,75.00


In [11]:
# Distribution of calculated metrics
q("""
    SELECT
        round(avg(trip_duration_minutes), 1) as avg_duration_min,
        round(median(trip_duration_minutes), 1) as median_duration_min,
        round(avg(avg_speed_mph), 1) as avg_speed,
        round(avg(tip_percentage), 1) as avg_tip_pct,
        round(avg(cost_per_mile), 2) as avg_cost_per_mile,
        count(CASE WHEN is_weekend THEN 1 END) as weekend_trips,
        count(CASE WHEN NOT is_weekend THEN 1 END) as weekday_trips
    FROM main_intermediate.int_trip_metrics
""")

,avg_duration_min,median_duration_min,avg_speed,avg_tip_pct,avg_cost_per_mile,weekend_trips,weekday_trips
0,15.0,12.0,11.4,23.1,8.87,743626,2160421


### Pre-Aggregation Models

Two more intermediate models aggregate trip-level data:

- **`int_daily_summary`** -- One row per day (31 rows for January). Total trips, revenue, averages.
- **`int_hourly_patterns`** -- One row per date + hour (~744 rows). For understanding demand curves.

In [12]:
# Daily summary -- what does January look like?
q("""
    SELECT
        pickup_date,
        pickup_day_of_week,
        is_weekend,
        total_trips,
        round(total_revenue, 0) as total_revenue,
        round(avg_trip_distance, 1) as avg_dist,
        round(avg_tip_percentage, 1) as avg_tip_pct
    FROM main_intermediate.int_daily_summary
    ORDER BY pickup_date
""")

,pickup_date,pickup_day_of_week,is_weekend,total_trips,total_revenue,avg_dist,avg_tip_pct
0,2024-01-01,Monday,False,78722,2428749.0,4.1,17.2
1,2024-01-02,Tuesday,False,73696,2275597.0,4.1,19.2
2,2024-01-03,Wednesday,False,80584,2355216.0,3.8,19.7
3,2024-01-04,Thursday,False,100893,2801040.0,3.3,20.4
4,2024-01-05,Friday,False,101106,2726894.0,3.2,26.1
5,2024-01-06,Saturday,True,95015,2428229.0,3.2,24.1
6,2024-01-07,Sunday,True,65811,1883934.0,3.9,20.3
7,2024-01-08,Monday,False,78362,2214412.0,3.5,20.9
8,2024-01-09,Tuesday,False,92263,2365014.0,2.9,21.6
9,2024-01-10,Wednesday,False,93342,2546556.0,3.2,21.6


### Dimensional Modeling (Kimball Methodology)

The marts layer follows **dimensional modeling**:

- **Fact tables** contain events/transactions with numeric measures (fares, distances, durations)
- **Dimension tables** contain descriptive attributes for filtering and grouping (locations, dates, payment types)

This design answers: *"Who took a trip, where, when, how did they pay, and how much did it cost?"*

```
                    dim_locations
                         |
int_trip_metrics --> fct_trips <-- dim_dates
                         |
                    dim_payment_types
```

### Why Not Just One Big Table?

You could, and for a small project it works fine. But dimensional modeling:
1. **Reduces redundancy** -- zone names stored once in `dim_locations`, not 2.9M times in the fact table
2. **Enables flexible analysis** -- join any dimension to the fact table
3. **Scales** -- when you add a new dimension (e.g., `dim_weather`), you only add one join
4. **Is the industry standard** -- every analytics engineer is expected to know this pattern

In [13]:
# Explore the dimension tables
print(f"dim_locations:      {q('SELECT count(*) as n FROM main_marts.dim_locations')['n'][0]} zones")
print(f"dim_dates:          {q('SELECT count(*) as n FROM main_marts.dim_dates')['n'][0]} days")
print(f"dim_payment_types:  {q('SELECT count(*) as n FROM main_marts.dim_payment_types')['n'][0]} types")

print("\n=== dim_dates (sample) ===")
display(q("SELECT * FROM main_marts.dim_dates LIMIT 7"))

print("\n=== dim_locations (top boroughs) ===")
display(q("""
    SELECT borough, count(*) as zones
    FROM main_marts.dim_locations
    GROUP BY borough ORDER BY zones DESC
"""))

dim_locations:      265 zones
dim_dates:          31 days
dim_payment_types:  6 types

=== dim_dates (sample) ===


,date_key,year,month,day_of_month,day_of_week_num,day_of_week_name,month_name,week_of_year,is_weekend,is_holiday
0,2024-01-01,2024,1,1,1,Monday,January,1,False,True
1,2024-01-02,2024,1,2,2,Tuesday,January,1,False,False
2,2024-01-03,2024,1,3,3,Wednesday,January,1,False,False
3,2024-01-04,2024,1,4,4,Thursday,January,1,False,False
4,2024-01-05,2024,1,5,5,Friday,January,1,False,False
5,2024-01-06,2024,1,6,6,Saturday,January,1,True,False
6,2024-01-07,2024,1,7,0,Sunday,January,1,True,False



=== dim_locations (top boroughs) ===


,borough,zones
0,Manhattan,69
1,Queens,69
2,Brooklyn,61
3,Bronx,43
4,Staten Island,20
5,EWR,1
6,Unknown,1
7,N/A,1


In [14]:
# The fact table -- trips enriched with location names
q("""
    SELECT
        trip_id,
        pickup_datetime,
        pickup_borough,
        pickup_zone,
        dropoff_borough,
        dropoff_zone,
        trip_duration_minutes,
        trip_distance_miles,
        fare_amount,
        tip_amount,
        total_amount
    FROM main_marts.fct_trips
    WHERE pickup_borough = 'Manhattan'
    LIMIT 10
""")

,trip_id,pickup_datetime,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone,trip_duration_minutes,trip_distance_miles,fare_amount,tip_amount,total_amount
0,c0b06c8ffd35a2cac5574970d8a36fd5,2024-01-10 03:35:38,Manhattan,Flatiron,Manhattan,Gramercy,6,1.61,9.3,0.00,14.30
1,bac7a292c9bf00a59b566a06ac941d7a,2024-01-10 03:42:25,Manhattan,Gramercy,Manhattan,Midtown East,3,1.03,6.5,1.00,12.50
2,b03163fcd1086af0b2cd01108c45176a,2024-01-10 03:24:39,Manhattan,East Village,Manhattan,Clinton East,11,2.60,14.2,0.00,19.20
3,133afb45eb3c52a76cf71410ffc6a37f,2024-01-10 03:38:30,Manhattan,East Chelsea,Manhattan,West Chelsea/Hudson Yards,5,0.73,6.5,2.00,13.50
4,27c9cfafe9675a735cedef3ffcbac891,2024-01-10 03:08:50,Manhattan,East Village,Manhattan,Two Bridges/Seward Park,6,0.94,7.9,3.23,16.13
5,b1f4b9657091e153b59aa792ed9a56d7,2024-01-10 03:52:57,Manhattan,Clinton East,Brooklyn,East Williamsburg,27,6.78,33.1,7.62,45.72
6,f56c25d29d6b30b151310c6640335b82,2024-01-10 03:46:15,Manhattan,West Chelsea/Hudson Yards,Manhattan,Clinton West,5,1.09,7.9,3.22,16.12
7,c2fd2793765853051e74ebc6dbba3e60,2024-01-10 03:19:14,Manhattan,Sutton Place/Turtle Bay North,Manhattan,UN/Turtle Bay South,2,0.99,6.5,0.00,11.50
8,0ec5bf8ecf646388276170d6c15d07dc,2024-01-10 03:20:58,Manhattan,Midtown Center,Manhattan,Meatpacking/West Village West,11,2.73,13.5,3.70,22.20
9,f576b3ebe13b873a79e8a8ccae53038d,2024-01-10 03:17:18,Manhattan,Union Sq,Queens,LaGuardia Airport,34,14.20,57.6,18.35,91.89


### Analytics Marts -- Pre-Aggregated for Speed

While the fact table supports any ad-hoc query, common business questions deserve **pre-computed answers**:

| Mart | Question It Answers |
|------|---------------------|
| `mart_daily_revenue` | How much revenue per day? What's the trend? |
| `mart_location_performance` | Which zones are most profitable? |
| `mart_hourly_demand` | When do people take taxis? Weekday vs weekend? |

In [15]:
# Daily revenue trend
daily = q("""
    SELECT
        date_key,
        day_of_week_name,
        is_weekend,
        total_trips,
        round(total_revenue, 0) as revenue,
        round(cumulative_revenue, 0) as cumulative,
        round(revenue_change_vs_prior_day, 0) as day_over_day
    FROM main_marts.mart_daily_revenue
    ORDER BY date_key
""")
daily

,date_key,day_of_week_name,is_weekend,total_trips,revenue,cumulative,day_over_day
0,2024-01-01,Monday,False,78722,2428749.0,2428749.0,NaN
1,2024-01-02,Tuesday,False,73696,2275597.0,4704346.0,-153151.0
2,2024-01-03,Wednesday,False,80584,2355216.0,7059562.0,79619.0
3,2024-01-04,Thursday,False,100893,2801040.0,9860603.0,445824.0
4,2024-01-05,Friday,False,101106,2726894.0,12587496.0,-74147.0
5,2024-01-06,Saturday,True,95015,2428229.0,15015725.0,-298665.0
6,2024-01-07,Sunday,True,65811,1883934.0,16899659.0,-544295.0
7,2024-01-08,Monday,False,78362,2214412.0,19114071.0,330478.0
8,2024-01-09,Tuesday,False,92263,2365014.0,21479085.0,150602.0
9,2024-01-10,Wednesday,False,93342,2546556.0,24025640.0,181542.0


In [16]:
# Top 15 pickup zones by total pickups
q("""
    SELECT
        pickup_borough,
        pickup_zone,
        total_pickups,
        round(total_revenue, 0) as total_revenue,
        round(avg_revenue_per_trip, 2) as avg_revenue,
        round(avg_tip_pct, 1) as avg_tip_pct,
        most_common_dropoff_zone,
        peak_pickup_hour
    FROM main_marts.mart_location_performance
    ORDER BY total_pickups DESC
    LIMIT 15
""")

,pickup_borough,pickup_zone,total_pickups,total_revenue,avg_revenue,avg_tip_pct,most_common_dropoff_zone,peak_pickup_hour
0,Manhattan,Midtown Center,141132,3376901.0,23.93,21.6,Upper East Side South,18
1,Manhattan,Upper East Side South,140793,2784629.0,19.78,22.8,Upper East Side North,14
2,Queens,JFK Airport,138349,11187589.0,80.86,14.2,Outside of NYC,16
3,Manhattan,Upper East Side North,134852,2732238.0,20.26,22.5,Upper East Side South,15
4,Manhattan,Midtown East,105010,2447523.0,23.31,21.9,Upper East Side South,18
5,Manhattan,Times Sq/Theatre District,103904,2789461.0,26.85,20.5,Penn Station/Madison Sq West,21
6,Manhattan,Penn Station/Madison Sq West,102807,2477827.0,24.10,20.9,Times Sq/Theatre District,18
7,Manhattan,Lincoln Square East,102518,2187210.0,21.33,22.5,Upper West Side South,22
8,Queens,LaGuardia Airport,87887,5824248.0,66.27,21.4,Times Sq/Theatre District,14
9,Manhattan,Upper West Side South,87327,1854244.0,21.23,22.7,Lincoln Square East,15


In [17]:
# Hourly demand: weekday vs weekend
q("""
    SELECT
        pickup_hour,
        CASE WHEN is_weekend THEN 'Weekend' ELSE 'Weekday' END as day_type,
        round(avg_trips_per_period, 0) as avg_trips,
        round(avg_revenue_per_period, 0) as avg_revenue,
        round(avg_distance, 1) as avg_distance,
        round(avg_duration_min, 1) as avg_duration
    FROM main_marts.mart_hourly_demand
    ORDER BY is_weekend, pickup_hour
""")

,pickup_hour,day_type,avg_trips,avg_revenue,avg_distance,avg_duration
0,0,Weekday,1660.0,53202.0,5.1,14.6
1,1,Weekday,945.0,28413.0,4.7,13.4
2,2,Weekday,616.0,17419.0,4.2,12.9
3,3,Weekday,452.0,13902.0,5.2,13.7
4,4,Weekday,390.0,14056.0,6.4,16.5
5,5,Weekday,646.0,23697.0,6.0,17.1
6,6,Weekday,1524.0,44733.0,4.5,16.1
7,7,Weekday,3205.0,83031.0,3.5,15.3
8,8,Weekday,4395.0,111503.0,3.0,15.6
9,9,Weekday,4533.0,118279.0,3.1,15.8


### Applying Week 5 to Any Dataset

**Intermediate layer rules of thumb:**
- If a calculation is used by 2+ downstream models, put it in intermediate
- Name intermediate models `int_*` -- they're stepping stones, not final products
- Materialize as views (they're intermediate, not queried directly by users)

**Dimensional modeling checklist:**
1. **Identify your facts**: What are the events/transactions? (trips, orders, clicks, etc.)
2. **Identify your dimensions**: What are the descriptive axes? (who, what, where, when, how)
3. **Build dimensions first**: They're smaller and simpler
4. **Join dimensions into the fact table** via foreign keys
5. **Add analytics marts** for common business questions
6. **Add contracts** to all mart models -- enforce column names and types at build time

**Model contracts** (dbt 1.5+) catch schema drift early:
```yaml
- name: fct_trips
  config:
    contract:
      enforced: true
  columns:
    - name: trip_id
      data_type: varchar
    - name: fare_amount
      data_type: decimal(10,2)
```

---

## Week 6: Advanced Features & Production Readiness

### Goals
- Implement incremental models (only process new data)
- Add Python models for statistical analysis
- Export data to parquet (external materialization)
- Track changes with snapshots (SCD Type 2)
- Define exposures for downstream consumers
- Set up CI/CD with GitHub Actions
- Generate and serve documentation

### What to Build
```bash
# 1. Convert fct_trips to incremental materialization
# 2. Create anomaly_daily_trips.py Python model
# 3. Create export_daily_revenue.sql (external materialization)
# 4. Create snap_locations.sql snapshot
# 5. Define exposures.yml
# 6. Set up .github/workflows/dbt-ci.yml

cd nyc_taxi_dbt
uv run dbt build --full-refresh --profiles-dir .  # First run: full refresh
uv run dbt build --profiles-dir .                  # Second run: incremental
uv run dbt snapshot --profiles-dir .               # Run snapshots
```

### Incremental Models: Only Process New Data

`fct_trips` is materialized as `incremental` with `delete+insert` strategy. On subsequent runs,
it only processes trips newer than what's already in the table.

```sql
-- fct_trips.sql (key part)
{{ config(
    materialized='incremental',
    unique_key='trip_id',
    incremental_strategy='delete+insert',
    on_schema_change='fail'
) }}

...
{% if is_incremental() %}
where t.pickup_datetime > (select max(pickup_datetime) from {{ this }})
{% endif %}
```

**Key detail**: When combining incremental + contracts, you must set `on_schema_change: 'fail'`
(or `'append_new_columns'`). Without it, dbt can't enforce the contract on incremental runs.

In [18]:
# fct_trips materialization type from dbt
fct_count = q("SELECT count(*) as n FROM main_marts.fct_trips")['n'][0]
print(f"fct_trips: {fct_count:,} rows")
print("\nOn a full refresh: processes all ~2.9M rows (~4 seconds)")
print("On incremental run: processes 0 new rows (~0.2 seconds)")
print("\nThis is the difference between 'rebuild everything' and 'process only changes'.")

fct_trips: 2,904,047 rows

On a full refresh: processes all ~2.9M rows (~4 seconds)
On incremental run: processes 0 new rows (~0.2 seconds)

This is the difference between 'rebuild everything' and 'process only changes'.


### Model Contracts: Enforce Schema at Build Time

All 7 mart models have **contracts enforced**. This means dbt checks the output schema
(column names and data types) at build time. If a model produces a column with the wrong type,
the build fails.

This catches bugs early -- if someone changes a column type upstream, the build breaks with a
clear error instead of silently corrupting data.

### Python Models: Anomaly Detection

`anomaly_daily_trips` is a native **dbt Python model** that uses pandas for statistical analysis.
It applies two anomaly detection methods:

1. **Z-score**: Flag days where trips/revenue are >2 standard deviations from the mean
2. **IQR (Interquartile Range)**: Flag days outside 1.5x the IQR -- more robust to outliers

**When to use Python over SQL**: Statistical functions, ML pipelines, or complex transformations
that are awkward in pure SQL.

**dbt-duckdb Python model gotcha**: `dbt.ref()` returns a DuckDB relation, not a table name.
Use `.df()` to convert to a pandas DataFrame.

In [19]:
# Anomaly detection results
anomalies = q("""
    SELECT
        pickup_date,
        pickup_day_of_week,
        is_weekend,
        total_trips,
        total_revenue,
        total_trips_z_score,
        total_revenue_z_score,
        is_anomaly
    FROM main_marts.anomaly_daily_trips
    ORDER BY total_trips_z_score ASC
""")

print(f"Total days analyzed: {len(anomalies)}")
print(f"Anomalous days: {anomalies['is_anomaly'].sum()}")

print("\n=== Flagged Anomalies ===")
display(anomalies[anomalies['is_anomaly'] == True])

print("\n=== Most Extreme Z-Scores (top 5 each direction) ===")
display(anomalies.head(5)[['pickup_date', 'pickup_day_of_week', 'total_trips', 'total_trips_z_score']])
display(anomalies.tail(5)[['pickup_date', 'pickup_day_of_week', 'total_trips', 'total_trips_z_score']])

Total days analyzed: 31
Anomalous days: 1

=== Flagged Anomalies ===


,pickup_date,pickup_day_of_week,is_weekend,total_trips,total_revenue,total_trips_z_score,total_revenue_z_score,is_anomaly
0,2024-01-07,Sunday,True,65811,1883933.99,-2.37,-2.52,True



=== Most Extreme Z-Scores (top 5 each direction) ===


,pickup_date,pickup_day_of_week,total_trips,total_trips_z_score
0,2024-01-07,Sunday,65811,-2.37
1,2024-01-02,Tuesday,73696,-1.70
2,2024-01-15,Monday,75367,-1.56
3,2024-01-08,Monday,78362,-1.30
4,2024-01-01,Monday,78722,-1.27


,pickup_date,pickup_day_of_week,total_trips,total_trips_z_score
26,2024-01-20,Saturday,106532,1.09
27,2024-01-17,Wednesday,108265,1.24
28,2024-01-25,Thursday,108303,1.24
29,2024-01-27,Saturday,108247,1.24
30,2024-01-18,Thursday,108324,1.25


### External Materialization: Parquet Export

`export_daily_revenue` uses dbt-duckdb's `external` materialization to write data directly
to a parquet file. This is useful when you need to share data with tools outside DuckDB
(Jupyter, Polars, Spark, S3 uploads).

```sql
{{ config(
    materialized='external',
    location='../exports/daily_revenue.parquet',
    format='parquet'
) }}
```

**Note**: The `exports/` target directory must exist before running. The external materialization
writes a parquet file and creates a DuckDB view that reads from it.

In [20]:
parquet_path = "exports/daily_revenue.parquet"
if os.path.exists(parquet_path):
    size_kb = os.path.getsize(parquet_path) / 1024
    # Read directly with DuckDB -- no need for the dbt-managed database
    exported = duckdb.query(f"SELECT * FROM read_parquet('{parquet_path}') LIMIT 5").fetchdf()
    rows = duckdb.query(f"SELECT count(*) FROM read_parquet('{parquet_path}')").fetchone()[0]
    print(f"Exported parquet: {parquet_path} ({size_kb:.1f} KB, {rows} rows)")
    print("\nThis file can be read by any tool that supports parquet:")
    print("  - pandas: pd.read_parquet('exports/daily_revenue.parquet')")
    print("  - polars: pl.read_parquet('exports/daily_revenue.parquet')")
    print("  - DuckDB: SELECT * FROM read_parquet('exports/daily_revenue.parquet')")
    display(exported)
else:
    print(f"File not found: {parquet_path}")
    print("Run: cd nyc_taxi_dbt && uv run dbt run --select export_daily_revenue --profiles-dir .")

Exported parquet: exports/daily_revenue.parquet (5.5 KB, 31 rows)

This file can be read by any tool that supports parquet:
  - pandas: pd.read_parquet('exports/daily_revenue.parquet')
  - polars: pl.read_parquet('exports/daily_revenue.parquet')
  - DuckDB: SELECT * FROM read_parquet('exports/daily_revenue.parquet')


,date_key,day_of_week_name,is_weekend,is_holiday,week_of_year,total_trips,total_passengers,total_fare_revenue,total_tip_revenue,total_revenue,avg_trip_revenue,avg_tip_percentage,credit_card_trips,cash_trips,avg_trip_distance,avg_trip_duration_min,cumulative_revenue,revenue_change_vs_prior_day
0,2024-01-01,Monday,False,True,1,78722,105959.0,1753039.09,259906.86,2428748.61,30.85,17.16,53050,14166,4.10,15.16,2428748.61,NaN
1,2024-01-02,Tuesday,False,False,1,73696,101420.0,1576987.85,262531.27,2275597.25,30.88,19.17,55380,15131,4.14,16.16,4704345.86,-153151.36
2,2024-01-03,Wednesday,False,False,1,80584,108452.0,1618091.03,274335.43,2355216.40,29.23,19.70,61645,15909,3.78,15.89,7059562.26,79619.15
3,2024-01-04,Thursday,False,False,1,100893,133110.0,1895034.94,336672.10,2801040.29,27.76,20.38,79318,17733,3.33,15.45,9860602.55,445823.89
4,2024-01-05,Friday,False,False,1,101106,134366.0,1837608.57,330024.94,2726893.76,26.97,26.14,79291,17449,3.20,14.80,12587496.31,-74146.53


### Snapshots: Tracking Changes Over Time (SCD Type 2)

`snap_locations` implements **SCD Type 2** (Slowly Changing Dimensions) for taxi zone definitions.
If a zone's borough or name changes, the old record gets a `dbt_valid_to` timestamp and a new
record is inserted.

In practice, taxi zones rarely change -- but the pattern is essential for production systems
tracking things like product prices, customer addresses, or employee roles.

```sql
{% snapshot snap_locations %}
{{ config(
    target_schema='snapshots',
    unique_key='location_id',
    strategy='check',
    check_cols=['borough', 'zone_name', 'service_zone'],
) }}
SELECT * FROM {{ ref('stg_taxi_zones') }}
{% endsnapshot %}
```

In [21]:
# Snapshot: SCD Type 2 columns
snap = q("""
    SELECT
        location_id,
        borough,
        zone_name,
        dbt_valid_from,
        dbt_valid_to,
        dbt_scd_id
    FROM snapshots.snap_locations
    ORDER BY location_id
    LIMIT 10
""")

total_snap = q("SELECT count(*) as n FROM snapshots.snap_locations")['n'][0]
current = q("SELECT count(*) as n FROM snapshots.snap_locations WHERE dbt_valid_to IS NULL")['n'][0]
historical = total_snap - current

print(f"Snapshot records: {total_snap} total ({current} current, {historical} historical)")
display(snap)

Snapshot records: 265 total (265 current, 0 historical)


,location_id,borough,zone_name,dbt_valid_from,dbt_valid_to,dbt_scd_id
0,1,EWR,Newark Airport,2026-02-12 11:45:48.503946,NaT,2aceed2a50760697167e755d44bca098
1,2,Queens,Jamaica Bay,2026-02-12 11:45:48.503946,NaT,6ca3b778aa1c9042500453006225cc3d
2,3,Bronx,Allerton/Pelham Gardens,2026-02-12 11:45:48.503946,NaT,f585be30008bd4a89f2d9b0e42b3dc00
3,4,Manhattan,Alphabet City,2026-02-12 11:45:48.503946,NaT,266b156411c0c0bc4775b80540a18a74
4,5,Staten Island,Arden Heights,2026-02-12 11:45:48.503946,NaT,6c7ce737c6b884ea71f9a6619ea9ee2b
5,6,Staten Island,Arrochar/Fort Wadsworth,2026-02-12 11:45:48.503946,NaT,29fff9a2e121b0420881dda1050ba6f6
6,7,Queens,Astoria,2026-02-12 11:45:48.503946,NaT,452e765e71fa205ee20a1d60fad23b70
7,8,Queens,Astoria Park,2026-02-12 11:45:48.503946,NaT,547d4c2384c4d4661b324e21386a2e7a
8,9,Queens,Auburndale,2026-02-12 11:45:48.503946,NaT,cf259f82e6af8ba9b52fa87728082d15
9,10,Queens,Baisley Park,2026-02-12 11:45:48.503946,NaT,481828be7134906c6bd0afb6e8e7cba5


### Exposures & Documentation

**Exposures** declare downstream consumers of your dbt models. They don't affect the build
but show up in the DAG and documentation:

```yaml
# exposures.yml
exposures:
  - name: nyc_taxi_analytics_dashboard
    type: dashboard
    maturity: medium
    depends_on:
      - ref('mart_daily_revenue')
      - ref('mart_hourly_demand')
      - ref('mart_location_performance')
```

**Documentation**: `dbt docs generate` creates a static site from your YAML descriptions.
`dbt docs serve` hosts it locally. The DAG view shows all model dependencies visually.

### CI/CD with GitHub Actions

Our `.github/workflows/dbt-ci.yml` runs on every PR and push to main:

```yaml
steps:
  - uses: actions/checkout@v4
  - name: Install uv
    uses: astral-sh/setup-uv@v4
  - run: uv python install 3.13
  - run: uv sync
  - run: uv run python scripts/download_data.py
  - run: uv run python scripts/setup_project.py
  - run: uv run dbt deps --profiles-dir .       # in nyc_taxi_dbt/
  - run: uv run dbt debug --profiles-dir .       # verify connection
  - run: uv run dbt build --full-refresh --profiles-dir .  # seed + run + test
  - run: uv run dbt docs generate --profiles-dir .
```

This ensures every code change is validated against real data before merging.

### The Full DAG

Here's how everything connects:

```
+--------------+     +------------------+     +------------------+     +---------------------------+
|  SOURCES     |     |    STAGING       |     |  INTERMEDIATE    |     |        MARTS              |
|              |     |                  |     |                  |     |                           |
| parquet -----+---->| stg_yellow_trips +---->| int_trip_metrics +---->| fct_trips (incremental)   |
|              |     |                  |     |     |            |     |   |                       |
| seeds:       |     | stg_taxi_zones --+-----+-----+------------+---->| dim_locations             |
| zone_lookup  |     | stg_payments ----+-----+-----+------------+---->| dim_payment_types         |
| payment_lkp  |     | stg_rate_codes   |     |     |            |     | dim_dates                 |
| rate_lkp     |     |                  |     |     v            |     |                           |
|              |     |                  |     | int_daily_summary+---->| mart_daily_revenue        |
|              |     |                  |     |     |            |     |   +-> export (parquet)    |
|              |     |                  |     |     +------------+---->| anomaly_daily (Python)    |
|              |     |                  |     | int_hourly_patt. +---->| mart_hourly_demand        |
|              |     |                  |     |                  |     |                           |
|              |     |                  |     |                  |     | mart_location_perf        |
+--------------+     +------------------+     +------------------+     +---------------------------+
```

**18 models. 91 tests. 1 snapshot. 2 exposures. Zero infrastructure.**

### Applying Week 6 to Any Dataset

**When to use incremental:**
- Your fact table has millions of rows and grows daily
- You have a reliable timestamp column for identifying new records
- Rebuilding from scratch takes too long for your update frequency

**When to add Python models:**
- Statistical analysis (z-scores, regressions, clustering)
- ML inference (running predictions on your data)
- Complex transformations that are painful in SQL

**When to add snapshots:**
- Any dimension that can change over time (prices, addresses, status codes)
- You need historical context ("what was the price when this order was placed?")

**Production readiness checklist:**
1. All mart models have contracts enforced
2. CI/CD runs `dbt build` on every PR
3. Source freshness checks are configured
4. Documentation is generated and accessible
5. Exposures declare downstream dependencies
6. A `Makefile` wraps common commands for the team

---

## Final Summary

In [22]:
# Final summary: what did we build?
print("=" * 60)
print("  PROJECT SUMMARY")
print("=" * 60)

# Note: export_daily_revenue is excluded because it's an external materialization view
# whose relative path (../exports/) only resolves correctly from nyc_taxi_dbt/, not repo root.
# It's explored in its own dedicated cell above.
tables = q("""
    SELECT table_schema, table_name, table_type
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
      AND table_name != 'export_daily_revenue'
    ORDER BY table_schema, table_name
""")

for schema in tables['table_schema'].unique():
    schema_tables = tables[tables['table_schema'] == schema]
    print(f"\n  {schema}:")
    for _, row in schema_tables.iterrows():
        count = q(f"SELECT count(*) as n FROM {schema}.{row['table_name']}")['n'][0]
        print(f"    {row['table_name']:<35} {count:>12,} rows  ({row['table_type']})")

print(f"\n  main_marts (external):")
print(f"    {'export_daily_revenue':<35} {'(parquet export -- see Week 6 cell above)'}")

print("\n" + "=" * 60)
total_tests = 91
print(f"  Tests: {total_tests} (84 data + 7 unit)")
print(f"  Materializations: view, table, incremental, external, python")
print(f"  Advanced: contracts, snapshots, exposures, source freshness")
print("=" * 60)

  PROJECT SUMMARY

  main:
    my_first_model                                 3 rows  (VIEW)
    my_second_model                                3 rows  (VIEW)

  main_intermediate:
    int_daily_summary                             31 rows  (VIEW)
    int_hourly_patterns                          744 rows  (VIEW)
    int_trip_metrics                       2,904,047 rows  (VIEW)

  main_marts:
    anomaly_daily_trips                           31 rows  (BASE TABLE)
    dim_dates                                     31 rows  (BASE TABLE)
    dim_locations                                265 rows  (BASE TABLE)
    dim_payment_types                              6 rows  (BASE TABLE)
    fct_trips                              2,904,047 rows  (BASE TABLE)
    fct_trips__dbt_backup                  2,904,047 rows  (BASE TABLE)
    mart_daily_revenue                            31 rows  (BASE TABLE)
    mart_hourly_demand                            48 rows  (BASE TABLE)
    mart_location_performance 

In [23]:
# Clean up
con.close()
print("Connection closed. Happy learning!")

Connection closed. Happy learning!


---

## Next Steps

Now that you understand the full pipeline, try these exercises:

1. **Add a new dimension**: Create `dim_rate_codes` from `stg_rate_codes` and join it into `fct_trips`
2. **Create a new mart**: `mart_borough_comparison` comparing metrics across boroughs
3. **Write a unit test**: Test that `int_daily_summary` correctly counts credit card vs cash trips
4. **Try the CLI tools**:
   ```bash
   make shell       # Explore the database interactively
   make validate    # Run the validation suite
   make benchmark   # See performance numbers
   make docs        # Browse the auto-generated documentation site
   ```
5. **Adapt to your own dataset**: Use the weekly "Applying to Any Dataset" sections as a checklist

## Resources

- [dbt Documentation](https://docs.getdbt.com/)
- [dbt Best Practices](https://docs.getdbt.com/best-practices/how-we-structure/1-guide-overview)
- [DuckDB Documentation](https://duckdb.org/docs/)
- [NYC TLC Trip Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

## Repository Structure

```
dbt_master/
  Makefile                    # Command shortcuts (make build, make test, etc.)
  pyproject.toml              # Python dependencies (uv sync)
  data/                       # Raw parquet files (gitignored)
  exports/                    # Parquet exports (gitignored)
  scripts/                    # Setup, download, validation, benchmarking
  docs/                       # Week-by-week learning notes
  .github/workflows/          # CI/CD pipeline
  nyc_taxi_dbt/
    dbt_project.yml           # Project configuration
    profiles.yml              # Connection config (gitignored)
    packages.yml              # dbt package dependencies
    models/
      sources.yml             # External source definitions
      exposures.yml           # Downstream consumer declarations
      example/                # Week 1 starter models
      staging/                # Clean + rename (4 models)
      intermediate/           # Calculate + aggregate (3 models)
      marts/core/             # Fact + dimension tables (4 models)
      marts/analytics/        # Business-level aggregations (5 models)
    seeds/                    # Reference CSVs (zones, payments, rates)
    macros/                   # Custom Jinja macros + tests
    snapshots/                # SCD Type 2 tracking
    tests/                    # Singular SQL tests
```